In [5]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical

# 设置路径
train_feature_dir = '../2025_A2/train/Features'
train_metadata_path = '../2025_A2/train/train_metadata.csv'

In [6]:

# 读取特征数据
color_df = pd.read_csv(os.path.join(train_feature_dir, 'color_histogram.csv')).drop(columns=['image_path'])
hog_df = pd.read_csv(os.path.join(train_feature_dir, 'hog_pca.csv')).drop(columns=['image_path'])
additional_df = pd.read_csv(os.path.join(train_feature_dir, 'additional_features.csv')).drop(columns=['image_path'])

# 合并特征
X = pd.concat([color_df, hog_df, additional_df], axis=1)
print(X.dtypes)  # 确保所有列都是 float 或 int 类型

ch_0            float64
ch_1            float64
ch_2            float64
ch_3            float64
ch_4            float64
                 ...   
hog_pca_19      float64
edge_density    float64
mean_b          float64
mean_g          float64
mean_r          float64
Length: 120, dtype: object


In [7]:
# 读取标签
metadata_df = pd.read_csv(train_metadata_path)
y = metadata_df['ClassId']
y_encoded = to_categorical(y, num_classes=43)

# 特征标准化
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 划分训练集与验证集
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y_encoded, test_size=0.2, random_state=42)

In [8]:
# 搭建神经网络模型
model = Sequential([
    Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.4),
    Dense(256, activation='relu'),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(43, activation='softmax')  # 43类
])

# 编译模型
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=25, batch_size=64)

Epoch 1/25
69/69 [==============================] - 1s 4ms/step - loss: 3.2987 - accuracy: 0.1528 - val_loss: 2.5752 - val_accuracy: 0.3634
Epoch 2/25
69/69 [==============================] - 0s 3ms/step - loss: 2.3121 - accuracy: 0.3827 - val_loss: 1.6015 - val_accuracy: 0.5692
Epoch 3/25
69/69 [==============================] - 0s 3ms/step - loss: 1.6637 - accuracy: 0.5068 - val_loss: 1.1611 - val_accuracy: 0.6512
Epoch 4/25
69/69 [==============================] - 0s 3ms/step - loss: 1.3320 - accuracy: 0.5765 - val_loss: 0.9758 - val_accuracy: 0.7022
Epoch 5/25
69/69 [==============================] - 0s 3ms/step - loss: 1.1271 - accuracy: 0.6360 - val_loss: 0.8543 - val_accuracy: 0.7231
Epoch 6/25
69/69 [==============================] - 0s 3ms/step - loss: 1.0042 - accuracy: 0.6713 - val_loss: 0.7805 - val_accuracy: 0.7568
Epoch 7/25
69/69 [==============================] - 0s 3ms/step - loss: 0.9031 - accuracy: 0.7036 - val_loss: 0.7375 - val_accuracy: 0.7523
Epoch 8/25
69/69 [==

In [11]:
# 评估验证集准确率
val_loss, val_acc = model.evaluate(X_val, y_val)
print(f"\nValidation Accuracy: {val_acc:.4f}")

# 保存模型（可选）
model.save('../models/neural_model.h5')

35/35 [==============================] - 0s 1ms/step - loss: 0.5254 - accuracy: 0.8397

Validation Accuracy: 0.8397
